# The AiiDA Graph Explorer (AGE)

## Introduction

The Aiida Graph Explorer is a tool that allows to query the AiiDA Graph. For simple (and not-so-simple) queries you can already use the QueryBuilder functionality.
The functionality shown here can help targeting recursive queries and operations that can be described as *Update rules*. Some examples are:
 - Get all nodes that are connected (via any kind of link) to a given node.
 - If groups are defined as *adjacent* if they store the same node, get all connected groups of a certain group

### First Example: Getting all children of a Node

In [1]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded

if not is_dbenv_loaded():
    load_dbenv(profile='test_dj1')

/home/leo/.virtualenvs/develop/local/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
import aiida
aiida.__file__

'/home/leo/git/AiiDA/aiida_develop_general/aiida/__init__.py'

In [16]:
from age.utils import create_tree
from aiida.utils.ascii_vis import draw_children
from aiida.common.links import  LinkType
# The number of layers I will create:
DEPTH = 4
# the branching at every level, i.e. the number of children per parent Node:
NR_OF_CHILDREN = 2

# Using a util function to create the tree:
parent, _, _ = create_tree(DEPTH, NR_OF_CHILDREN)
# Using a visualizer within AiiDA!
print(draw_children(parent, dist=DEPTH,
        follow_links_of_type=(LinkType.INPUT_CALC, LinkType.CREATE)))


                                                  /-CalculationNode [20208]
                                      /Data [20204]
                                     |            \-CalculationNode [20209]
               /CalculationNode [20202]
              |                      |            /-CalculationNode [20210]
              |                       \Data [20205]
              |                                   \-CalculationNode [20211]
-- /Data [20201]
              |                                   /-CalculationNode [20212]
              |                       /Data [20206]
              |                      |            \-CalculationNode [20213]
               \CalculationNode [20203]
                                     |            /-CalculationNode [20214]
                                      \Data [20207]
                                                  \-CalculationNode [20215]


In [ ]:
        # Created all the nodes, tree. 
        #Now testing whether I find all the descendants
        # Using the utility function to create the starting entity set:
        es = get_entity_sets(node_ids=(parent.id,))
        qb = QueryBuilder().append(Node).append(Node)

        for depth in range(0, self.DEPTH):
            #print('At depth {}'.format(depth))

            rule = UpdateRule(qb, mode=MODES.REPLACE, max_iterations=depth)
            res = rule.run(es.copy())['nodes']._set
            #print('   Replace-mode results: {}'.format(', '.join(map(str, sorted(res)))))
            should_set = desc_dict[depth]
            self.assertTrue(not(res.difference(should_set) or should_set.difference(res)))

            rule = UpdateRule(qb, mode=MODES.APPEND, max_iterations=depth)
            res = rule.run(es.copy())['nodes']._set
            #print('   Append-mode  results: {}'.format(', '.join(map(str, sorted(res)))))
            should_set = set()
            [[should_set.add(s) for s in desc_dict[d]] for d in range(depth+1)]

            self.assertTrue(not(res.difference(should_set) or should_set.difference(res)))
